<a href="https://colab.research.google.com/github/mdsayem01k/ML/blob/main/linera_R_Label_football.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Use label encoding**

In [16]:
#import necessary packeage
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score

In [17]:
#import dataset
data = pd.read_pickle("/content/updated_football_data_new_match_november.pkl")
data.head()

,id,id_game,country,competition,round,date,home,away,ft_home,ft_away,...,over 2.5 yellow_cards_2nd_half,over 3.5 yellow_cards_2nd_half,over 4.5 yellow_cards_2nd_half,1st_home_BallP_domination,1st_away_BallP_domination,1st_home_domination,1st_away_domination,1st_total_domination,1st_dominator,season
0,0,zyA2QCuE,Germany,2. Bundesliga,5,03.09.2023 13:30,Greuther Furth,Hannover,1,3,...,1,0,0,0,0,6,3,9,1,2023
1,1,YcA45B3l,Germany,2. Bundesliga,5,03.09.2023 13:30,Hamburger SV,Hansa Rostock,2,0,...,1,1,1,1,0,10,3,13,11,2023
2,2,djbBOjAQ,Germany,2. Bundesliga,5,03.09.2023 13:30,VfL Osnabruck,Elversberg,0,1,...,0,0,0,0,1,4,1,5,11,2023
3,3,8UHG29m7,Germany,2. Bundesliga,5,02.09.2023 20:30,Kaiserslautern,Nurnberg,3,1,...,1,0,0,0,1,4,8,12,2,2023
4,4,Q5SB3kY0,Germany,2. Bundesliga,5,02.09.2023 13:00,Holstein Kiel,Paderborn,2,1,...,1,0,0,0,0,4,5,9,2,2023


In [18]:
#select input features
#competition,round--->data.columns[3:5]
#data.columns[10:43]
#1st_winner-->data.coulumns[61]
# goals_1st_half-->data.columns[68:74]
#1st_half_corner--> data.columns[84:89]
# 1st_half_Yellow_card--> data.columns[99:104]
# data.columns[109:]


In [40]:
features_data=data.iloc[:, list(range(3,5))+[6,7] + list(range(10, 43))+list(range(68, 74))+list(range(84,89))+list(range(99, 104))+list(range(109, 116))]
target_home=data.iloc[:,8]

In [41]:
#target_home for target output for home team
target_home=data["ft_home"]
#target_away for target output for away team
target_away=data['ft_away']

In [42]:
#check missing values
features_missing_values=features_data.isnull().sum()
features_missing_values[:20]

competition             625
round                  5791
home                      0
away                      0
team_first_goal           0
time_first_goal           0
timeband_first_goal       0
odds_home                36
odds_draw               804
odds_away               804
favorite                  0
home_BallP               41
away_BallP               41
home_Shoot               18
away_Shoot               18
home_SOT                 18
away_SOT                 18
home_Corn                18
away_Corn                18
yc_home                   0
dtype: int64

In [43]:
# replace all NA's the value that comes directly after it in the same column,
# then replace all the remaining na's with 0
features_data=features_data.fillna(method='bfill', axis=0,).fillna(0)
features_data.isnull().sum()

competition                       0
round                             0
home                              0
away                              0
team_first_goal                   0
time_first_goal                   0
timeband_first_goal               0
odds_home                         0
odds_draw                         0
odds_away                         0
favorite                          0
home_BallP                        0
away_BallP                        0
home_Shoot                        0
away_Shoot                        0
home_SOT                          0
away_SOT                          0
home_Corn                         0
away_Corn                         0
yc_home                           0
yc_away                           0
rc_home                           0
rc_away                           0
1st_home                          0
1st_away                          0
1st_home_BallP                    0
1st_away_BallP                    0
1st_home_Shoot              

In [44]:
from sklearn.preprocessing import LabelEncoder

def label_encode_columns(df, columns):
    df_encoded = df.copy()
    label_encoder = LabelEncoder()

    for column in columns:
        # Handle missing values by filling with a placeholder
        df_encoded[column].fillna("missing", inplace=True)

        # Apply label encoding
        df_encoded[column] = label_encoder.fit_transform(df_encoded[column])

    return df_encoded

In [45]:
object_columns = features_data.select_dtypes(include='object').columns.tolist()
object_columns
features_data=label_encode_columns(features_data,object_columns)

In [46]:
features_data

,competition,round,home,away,team_first_goal,time_first_goal,timeband_first_goal,odds_home,odds_draw,odds_away,...,over 2.5 yellow_cards_1st_half,over 3.5 yellow_cards_1st_half,over 4.5 yellow_cards_1st_half,1st_home_BallP_domination,1st_away_BallP_domination,1st_home_domination,1st_away_domination,1st_total_domination,1st_dominator,season
0,0,41,410,434,2,14,2,1.96,3.65,3.60,...,0,0,0,0,0,23,17,31,1,21
1,0,41,428,435,1,53,4,1.48,4.65,6.02,...,1,0,0,1,0,2,17,5,11,21
2,0,41,1085,306,2,83,6,2.30,3.77,2.75,...,0,0,0,0,1,21,1,27,11,21
3,0,41,508,722,1,17,2,2.12,3.48,3.38,...,0,0,0,0,1,21,22,4,2,21
4,0,41,454,758,2,8,1,2.25,3.77,2.86,...,1,0,0,0,0,21,19,31,2,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108503,42,0,1143,569,1,53,4,1.38,5.05,6.90,...,0,0,0,0,0,25,5,14,2,0
108504,42,0,1164,1147,1,26,2,1.49,4.40,6.02,...,0,0,0,1,0,23,17,31,1,0
108505,42,0,1126,620,0,0,0,3.68,3.60,1.92,...,1,1,1,0,0,1,18,27,22,0
108506,42,0,407,913,2,14,2,3.40,3.62,2.02,...,0,0,0,0,0,21,19,31,2,0


# **Build model for Home Team**




In [27]:
X_train,X_test,y_train,y_test = train_test_split(features_data,target_home,
                                                test_size=0.2)

In [28]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [29]:
y_pred=model.predict(X_test)
y_pred

array([1.6996564 , 0.60372714, 2.34913866, ..., 0.37940418, 1.28153504,
       0.8239164 ])

In [30]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Report for Home Team')
print(f'Mean Squared Error: {mse}')
print(f'R-squared (R2) Score: {r2}')

Report for Home Team
Mean Squared Error: 0.7108892785991604
R-squared (R2) Score: 0.5636893058357417


# **Build Model for Away Team**

In [31]:
X_train,X_test,y_train,y_test = train_test_split(features_data,target_away,
                                                test_size=0.2)

In [32]:
model_ft_away = LinearRegression()
model_ft_away.fit(X_train, y_train)

LinearRegression()

In [33]:
y_pred_ft_awar=model.predict(X_test)

In [34]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred_ft_awar)
r2 = r2_score(y_test, y_pred_ft_awar)
print(f"Report for Away team")
print(f'Mean Squared Error: {mse}')
print(f'R-squared (R2) Score: {r2}')

Report for Away team
Mean Squared Error: 2.2697484438066664
R-squared (R2) Score: -0.7541500706741697


# **Scaling**
Home team



In [48]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the features
features = scaler.fit_transform(features_data)

# Create a DataFrame with the standardized features
features = pd.DataFrame(features, columns=features_data.columns)


In [50]:
features.head(5)

,competition,round,home,away,team_first_goal,time_first_goal,timeband_first_goal,odds_home,odds_draw,odds_away,...,over 2.5 yellow_cards_1st_half,over 3.5 yellow_cards_1st_half,over 4.5 yellow_cards_1st_half,1st_home_BallP_domination,1st_away_BallP_domination,1st_home_domination,1st_away_domination,1st_total_domination,1st_dominator,season
0,-1.820455,1.613753,-0.475753,-0.409073,1.105965,-0.883673,-0.237588,-0.357313,-0.044912,-0.142639,...,-0.643376,-0.384094,-0.233868,-0.66547,-0.517864,0.629477,0.314309,1.159137,-0.711979,1.534154
1,-1.820455,1.613753,-0.422047,-0.406095,-0.528750,0.563307,0.995236,-0.642157,0.916799,0.690906,...,1.554302,-0.384094,-0.233868,0.45292,-0.517864,-2.006280,0.314309,-0.940738,0.682337,1.534154
2,-1.820455,1.613753,1.538244,-0.790226,1.105965,1.676368,2.228059,-0.155549,0.070493,-0.435413,...,-0.643376,-0.384094,-0.233868,-0.66547,0.810719,0.378452,-1.973754,0.836080,0.682337,1.534154
3,-1.820455,1.613753,-0.183351,0.448523,-0.528750,-0.772367,-0.237588,-0.262365,-0.208403,-0.218415,...,-0.643376,-0.384094,-0.233868,-0.66547,0.810719,0.378452,1.029328,-1.021503,-0.572548,1.534154
4,-1.820455,1.613753,-0.344470,0.555723,1.105965,-1.106285,-0.854000,-0.185220,0.070493,-0.397524,...,1.554302,-0.384094,-0.233868,-0.66547,-0.517864,0.378452,0.600317,1.159137,-0.572548,1.534154


In [51]:
X_train,X_test,y_train,y_test = train_test_split(features_data,target_home,
                                                test_size=0.2)

In [52]:
model_ft_away = LinearRegression()
model_ft_away.fit(X_train, y_train)

LinearRegression()

In [53]:
y_pred_ft_awar=model.predict(X_test)

In [56]:
from sklearn.metrics import mean_squared_error, r2_score
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Report for Home Team')
print(f'Mean Squared Error: {mse}')
print(f'R-squared (R2) Score: {r2}')

Report for Home Team
Mean Squared Error: 2.521194142173915
R-squared (R2) Score: -0.5695866023560217
